In [1]:
DATA_NAME = 'agriVision-full' 
TRANSFORM = 'wavelet-horizVert'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_horizVert_wavelet_full_agriVision.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,1696955.02,-23.61,-12.55,-7.22,-0.79,-0.78,0.24,-0.18,-0.00
3,-23.61,340404.97,-4.30,0.88,0.39,-0.03,-0.04,0.02,0.02
4,-12.55,-4.30,75414.71,0.29,0.43,-0.04,-0.04,-0.00,-0.00
5,-7.22,0.88,0.29,10815.36,-0.06,-0.00,0.00,-0.02,-0.00
6,-0.79,0.39,0.43,-0.06,1955.56,0.03,0.01,0.00,0.00
7,-0.78,-0.03,-0.04,-0.00,0.03,394.76,0.00,0.00,-0.00
8,0.24,-0.04,-0.04,0.00,0.01,0.00,99.86,0.00,0.00
9,-0.18,0.02,-0.00,-0.02,0.00,0.00,0.00,22.46,-0.00
10,-0.00,0.02,-0.00,-0.00,0.00,-0.00,0.00,-0.00,1.88


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00003,-0.00004,-0.00005,-0.00001,-0.00003,0.00002,-0.00003,-0.00000
3,-0.00003,1.00000,-0.00003,0.00001,0.00002,-0.00000,-0.00001,0.00001,0.00002
4,-0.00004,-0.00003,1.00000,0.00001,0.00004,-0.00001,-0.00002,-0.00000,-0.00000
5,-0.00005,0.00001,0.00001,1.00000,-0.00001,-0.00000,0.00000,-0.00003,-0.00002
6,-0.00001,0.00002,0.00004,-0.00001,1.00000,0.00003,0.00001,0.00002,0.00000
7,-0.00003,-0.00000,-0.00001,-0.00000,0.00003,1.00000,0.00001,0.00005,-0.00000
8,0.00002,-0.00001,-0.00002,0.00000,0.00001,0.00001,1.00000,0.00001,0.00000
9,-0.00003,0.00001,-0.00000,-0.00003,0.00002,0.00005,0.00001,1.00000,-0.00003
10,-0.00000,0.00002,-0.00000,-0.00002,0.00000,-0.00000,0.00000,-0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

39.69930837284727

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.20183678e+11 1.26793189e+10 6.09062436e+08 1.21944062e+07
 3.82978892e+05 1.46867005e+04 8.35862949e+02 3.13118041e+01
 1.49588317e-31]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999643,-0.026108,-0.005538,-0.000796,-0.000144,-0.000029,-0.000007,-0.000002,-1.398901e-07
1,0.025905,0.999096,-0.033401,-0.004522,-0.000816,-0.000165,-0.000042,-0.000009,-7.371468e-07
2,0.006380,0.033122,0.999120,-0.024547,-0.004330,-0.000875,-0.000222,-0.000050,-4.167142e-06
3,0.001061,0.005271,0.024193,0.998964,-0.037425,-0.007270,-0.001836,-0.000414,-3.478019e-05
4,0.000229,0.001139,0.005141,0.036844,0.997834,-0.052650,-0.012684,-0.002843,-2.377523e-04
5,0.000057,0.000284,0.001281,0.008940,0.050941,0.994451,-0.089698,-0.018529,-1.544261e-03
6,0.000019,0.000096,0.000432,0.003008,0.016664,0.086115,0.989064,-0.118178,-9.144499e-03
7,0.000007,0.000033,0.000148,0.001035,0.005716,0.028359,0.114852,0.989311,-8.506094e-02
8,0.000001,0.000005,0.000025,0.000173,0.000957,0.004740,0.018741,0.083347,9.963326e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00035654, 0.00090434, 0.00088033, 0.00103614, 0.00216551,
       0.00554946, 0.01093593, 0.01068929, 0.00366744])